In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from time import time
from ComCheb import mpRemez,Contours,mpRoots
from mpmath import mp
mp.pretty = 1


storage = dict()

In [ ]:
with open('Data/roots.pkl','rb') as fp:
    storage = pickle.load(fp)

In [ ]:
ToDo = [('P',3,16),('P',3,62),
        ('P',4,17),('P',4,82),
        ('P',5,21),('P',5,102),
        ('P',6,27),('P',6,122),
        ('C',0.5,11),('C',0.5,21),
        ('C',1.5,10),('C',1.5,50),
        ('H',3,31),('H',3,47),
        ('H',4,41),('H',4,62),
        ('H',5,51),('H',5,77),
        ('H',6,61),('H',6,92),
        ('L',1,11),('L',1,41),
        ('L',2,11),('L',2,41),
        ('E3',1,25),('E3',1,40),
        ('E4',5,21),('E4',5,37)]

def poly_cap(m,sl=1):   # mpmath implementation of capacity of polygons
    m = mp.mpf(m)
    return mp.gamma(1/m)/(2**(1+2/m)*mp.pi**0.5*mp.gamma(1/2+1/m))
curves = {'P':lambda m: Contours.mpPolygon(m,sl=1/poly_cap(m)),
          'C':Contours.mpCircLune,
          'H':Contours.mpHypocycloid,
          'L':lambda r : Contours.mpLemniscate(r=r),
          'E3':lambda var: Contours.mpE3(var),
          'E4':lambda r,maxsteps=10000 : Contours.mpE4(r,maxsteps)}

In [ ]:
mp.dps= 50
prec  = 1e-30

for task in ToDo:
    t1 = time()
    if task in storage.keys():
        t,a = storage[task][:2]
    else:
        t,a = None,None
    g,m,n = task
    gamma = curves[g](m)

    if g =='L' or g=='C':
        symm = 2
    elif g=='E3':
        symm = 1
    elif g=='E4':
        symm = 2
    else:
        symm = m

    t,a,L,h,_ = mpRemez(gamma,n,t=t,a=a,symmetry=symm,rc=1,
                        prec=prec,Mmax=1000,reps=30,plot=0,pinfo=0,samples=1001)
    r = mpRoots(L,n,symm)
    storage[task] = (t,a,L,h,r)
    print(task,'\t',int(time()-t1))


with open('Data/roots.pkl','wb') as fp:
    pickle.dump(storage,fp)
